In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

### 전처리된 데이터 불러오기

In [3]:
baby_growth_df = pd.read_csv("./baby_growth_df.csv")
baby_growth_df

,days,height,weight,gender
0,0,52.0,3.78,1
1,0,52.0,3.78,1
2,0,50.0,3.20,1
3,1,51.0,2.45,1
4,1,50.0,2.80,1
...,...,...,...,...
79344,365,74.6,9.60,0
79345,365,74.6,8.60,0
79346,365,74.6,9.40,0
79347,365,74.6,9.40,0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### height 예측 모델링

In [5]:
# 독립 변수와 종속 변수 분리
X = baby_growth_df[['days','gender']]
y = baby_growth_df['height']

# train/test 데이터 분리, 성별 비중 유지
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=X['gender'], random_state=42)

### (1) LinearRegression - height 예측 모델

In [6]:
# linear regression 모델 학습
lr_height = LinearRegression()
lr_height.fit(X_train, y_train)

# test 데이터로 예측
y_pred = lr_height.predict(X_test)

# 모델 성능 평가
r2 = r2_score(y_test, y_pred)
print('R-squared:', r2)

# MSE 평가
mse = mean_squared_error(y_test, y_pred)
print("LinearRegression MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lr_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr_height.score(X_test, y_test)))

R-squared: 0.9053437089619766
LinearRegression MSE: 3.528196169173679
훈련 세트 정확도: 0.905
테스트 세트 정확도: 0.905


### (1-1) 이차항 선형 회귀 - height 예측 모델

In [140]:
# 이차항 생성
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_height = poly.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly_height, y, test_size=0.25, stratify=X['gender'], random_state=42)

# 모델 학습
lr2_height = LinearRegression()
lr2_height.fit(X_train, y_train)

# 예측
y_pred = lr2_height.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred)
print("linearRegression2 Mean Squared Error:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lr2_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr2_height.score(X_test, y_test)))


linearRegression2 Mean Squared Error: 0.6987426294820096
훈련 세트 정확도: 0.981
테스트 세트 정확도: 0.981


### (2) RandomForestRegressor - height 예측 모델

In [92]:
# RandomForestRegressor 모델 객체 생성
rf_height = RandomForestRegressor(random_state=42)

# 모델 학습
rf_height.fit(X_train, y_train)

# 모델 평가 - 예측값 생성
y_pred = rf_height.predict(X_test)

# 모델 평가 - 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print("RandomForestRegressor Mean Squared Error:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(rf_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_height.score(X_test, y_test)))

RandomForestRegressor Mean Squared Error: 0.25375292488684353
훈련 세트 정확도: 0.993
테스트 세트 정확도: 0.993


### (3) XGBRegressor - height 예측 모델

In [93]:
# 모델 객체 생성
xgb_height = XGBRegressor(objective='reg:squarederror', tree_method='hist', random_state=42)

# 모델 학습
xgb_height.fit(X_train, y_train)

# 검증용 데이터를 사용하여 예측 수행
y_pred = xgb_height.predict(X_test)

# 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"xgboost MSE: {mse:.3f}")
print(f"xgboost MAE: {mae:.3f}")
print(f"xgboost R-squared: {r2:.3f}")

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(xgb_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(xgb_height.score(X_test, y_test)))

xgboost MSE: 0.316
xgboost MAE: 0.204
xgboost R-squared: 0.992
훈련 세트 정확도: 0.992
테스트 세트 정확도: 0.992


### RandomForestRegressor hyperparameter tuning

In [9]:
# 하이퍼파라미터 후보 값들
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV 수행
rf_height = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_height, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 그 때의 성능 출력
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV score:", -grid_search.best_score_)

# 최적의 하이퍼파라미터로 모델 학습 및 평가
rf_height_best = RandomForestRegressor(n_estimators=grid_search.best_params_['n_estimators'],
                                 max_depth=grid_search.best_params_['max_depth'],
                                 min_samples_split=grid_search.best_params_['min_samples_split'],
                                 random_state=42)
rf_height_best.fit(X_train, y_train)
y_pred_rf = rf_height_best.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred_rf))

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(rf_height_best.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_height_best.score(X_test, y_test)))


Best hyperparameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}
Best CV score: 0.2536826197356331
Test MSE: 0.2537769441848588
훈련 세트 정확도: 0.993
테스트 세트 정확도: 0.993


### (4) LightGBM - height 예측 모델

In [159]:
# LightGBM 모델링
lgbm_height = LGBMRegressor(random_state=42)
lgbm_height.fit(X_train, y_train)

# 예측값 생성
y_pred = lgbm_height.predict(X_test)

# MSE 평가
mse = mean_squared_error(y_test, y_pred)
print("lightGBM MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lgbm_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lgbm_height.score(X_test, y_test)))

lightGBM MSE: 0.30606518685357387
훈련 세트 정확도: 0.992
테스트 세트 정확도: 0.992


### (5) GradientBoosting - height 예측 모델

In [95]:
# Gradient Boosting 모델 학습
gbr_height = GradientBoostingRegressor(random_state=42)
gbr_height.fit(X_train, y_train)

# 모델 예측
y_pred = gbr_height.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
print("GradientBoostingRegressor MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(gbr_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbr_height.score(X_test, y_test)))

GradientBoostingRegressor MSE: 0.35741078812332194
훈련 세트 정확도: 0.990
테스트 세트 정확도: 0.990


### Weight 예측 모델링

In [7]:
# 독립 변수와 종속 변수 분리
X_data = baby_growth_df[['days','gender','height']]
y_target = baby_growth_df['weight']

# train/test 데이터 분리, 성별 비중 유지
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.25, stratify=X_data['gender'], random_state=42)

In [146]:
# 이차항 생성
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_weight = poly.fit_transform(X_data)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly_weight, y_target, test_size=0.25, stratify=X_data['gender'], random_state=42)

# 모델 학습
lr2_weight = LinearRegression()
lr2_weight.fit(X_train, y_train)

# 예측
y_pred = lr2_weight.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred)
print("linearRegression2 Mean Squared Error:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lr2_weight.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr2_weight.score(X_test, y_test)))


linearRegression2 Mean Squared Error: 0.3476746801007111
훈련 세트 정확도: 0.890
테스트 세트 정확도: 0.890


### (1) RandomForestRegressor - weight 예측 모델

In [47]:
# 하이퍼파라미터 후보 값들
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV 수행
rf_weight = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_weight, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 그 때의 성능 출력
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV score:", -grid_search.best_score_)

# 최적의 하이퍼파라미터로 모델 학습 및 평가
rf_weight_best = RandomForestRegressor(n_estimators=grid_search.best_params_['n_estimators'],
                                 max_depth=grid_search.best_params_['max_depth'],
                                 min_samples_split=grid_search.best_params_['min_samples_split'],
                                 random_state=42)
rf_weight_best.fit(X_train, y_train)
y_pred_rf = rf_weight_best.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred_rf))

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(rf_weight_best.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_weight_best.score(X_test, y_test)))


Best hyperparameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Best CV score: 0.3465750289982933
Test MSE: 0.34468287750259985
훈련 세트 정확도: 0.894
테스트 세트 정확도: 0.891


AttributeError: 'RandomForestRegressor' object has no attribute 'oob_score_'

### (2) LinearRegressor - weight 예측 모델

In [8]:
lr_weight = LinearRegression()

lr_weight.fit(X_train, y_train)
lr_weight.score(X_test, y_test)

print("훈련 세트 정확도: {:.3f}".format(lr_weight.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr_weight.score(X_test, y_test)))

훈련 세트 정확도: 0.885
테스트 세트 정확도: 0.885


### (3) GradientBoostingRegressor - weight 예측 모델

In [98]:
gbr_weight = GradientBoostingRegressor()

gbr_weight.fit(X_train, y_train)

# 평가
print("훈련 세트 정확도: {:.3f}".format(gbr_weight.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbr_weight.score(X_test, y_test)))

훈련 세트 정확도: 0.892
테스트 세트 정확도: 0.891


### (4) lightGBM - weight 예측 모델

In [161]:
# LightGBM 모델링
lgbm_weight = LGBMRegressor(random_state=42)
lgbm_weight.fit(X_train, y_train)

# 예측값 생성
y_pred = lgbm_weight.predict(X_test)

# MSE 평가
mse = mean_squared_error(y_test, y_pred)
print("lightGBM MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lgbm_weight.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lgbm_weight.score(X_test, y_test)))

lightGBM MSE: 0.34462365868841655
훈련 세트 정확도: 0.893
테스트 세트 정확도: 0.891


In [9]:
# 사용자로부터 데이터 입력받기
input_days = int(input("생후 일 수를 입력하세요: "))
input_height = float(input("아기의 현재 키를 입력하세요 (단위: cm): "))
input_weight = float(input("아기의 현재 몸무게를 입력하세요 (단위: kg): "))
input_gender = input("아기의 성별을 입력하세요 (F/M): ")

# 입력받은 데이터로 1개월, 3개월, 6개월 뒤의 키 예측
if input_gender == 'F':
    gender = 0
elif input_gender == 'M':
    gender = 1
else:
    print("성별은 'F' 또는 'M' 중에서 입력해주세요.")
    exit()


# polynomial regression
#one_month_pred_height = lr2_height.predict([[(input_days+30)**2, (input_days + 30), gender]])
#three_month_pred_height = lr2_height.predict(np.array([[((input_days+90)**2), (input_days+90), gender]])) 
#six_month_pred_height = lr2_height.predict(np.array([[((input_days+180)**2), (input_days+180), gender]])) 

#one_month_pred_weight = lr2_weight.predict(np.array([[((input_days+30)**2), (input_days+30), gender, one_month_pred_height]]))
#three_month_pred_weight = lr2_weight.predict(np.array([[((input_days+90)**2), (input_days+90), gender, three_month_pred_height]])) 
#six_month_pred_weight = lr2_weight.predict(np.array([[((input_days+180)**2), (input_days+180), gender, six_month_pred_height]])) 
one_month_pred_height = lr_height.predict([[input_days+30, gender]])
three_month_pred_height = lr_height.predict([[input_days+90, gender]])
six_month_pred_height = lr_height.predict([[input_days+180, gender]])

one_month_pred_weight = lr_weight.predict([[input_days+30, gender, one_month_pred_height]])
three_month_pred_weight = lr_weight.predict([[input_days+90, gender, three_month_pred_height]])
six_month_pred_weight = lr_weight.predict([[input_days+180, gender, six_month_pred_height]])


# 결과 출력
print(f"아기의 1개월 뒤 예상 키는 {one_month_pred_height[0]:.1f}cm 입니다.")
print(f"아기의 3개월 뒤 예상 키는 {three_month_pred_height[0]:.1f}cm 입니다.")
print(f"아기의 6개월 뒤 예상 키는 {six_month_pred_height[0]:.1f}cm 입니다.")
print(f"아기의 1개월 뒤 예상 몸무게는 {one_month_pred_weight[0]:.1f}kg 입니다.")
print(f"아기의 3개월 뒤 예상 몸무게는 {three_month_pred_weight[0]:.1f}kg 입니다.")
print(f"아기의 6개월 뒤 예상 몸무게는 {six_month_pred_weight[0]:.1f}kg 입니다.")

생후 일 수를 입력하세요: 70
아기의 현재 키를 입력하세요 (단위: cm): 68
아기의 현재 몸무게를 입력하세요 (단위: kg): 4.8
아기의 성별을 입력하세요 (F/M): F
아기의 1개월 뒤 예상 키는 60.3cm 입니다.
아기의 3개월 뒤 예상 키는 65.3cm 입니다.
아기의 6개월 뒤 예상 키는 72.7cm 입니다.
아기의 1개월 뒤 예상 몸무게는 6.1kg 입니다.
아기의 3개월 뒤 예상 몸무게는 7.4kg 입니다.
아기의 6개월 뒤 예상 몸무게는 9.4kg 입니다.


In [10]:
# 독립 변수와 종속 변수 분리
X = baby_growth_df[['days','gender']]
y = baby_growth_df['height']

# train/test 데이터 분리, 성별 비중 유지
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=X['gender'], random_state=42)

In [11]:
# for height graph
graph_height = [input_height] # 0일 뒤 키 == 현재 입력 키
x_height_days = list(range(1,181)) # 1일 뒤부터 180일 뒤 까지
for i in range(1,181):
    graph_height.append(round(lr_height.predict([[input_days + i, gender]])[0],2))
graph_height


[68.0,
 57.9,
 57.98,
 58.06,
 58.15,
 58.23,
 58.31,
 58.39,
 58.48,
 58.56,
 58.64,
 58.72,
 58.81,
 58.89,
 58.97,
 59.06,
 59.14,
 59.22,
 59.3,
 59.39,
 59.47,
 59.55,
 59.63,
 59.72,
 59.8,
 59.88,
 59.97,
 60.05,
 60.13,
 60.21,
 60.3,
 60.38,
 60.46,
 60.54,
 60.63,
 60.71,
 60.79,
 60.88,
 60.96,
 61.04,
 61.12,
 61.21,
 61.29,
 61.37,
 61.45,
 61.54,
 61.62,
 61.7,
 61.79,
 61.87,
 61.95,
 62.03,
 62.12,
 62.2,
 62.28,
 62.36,
 62.45,
 62.53,
 62.61,
 62.7,
 62.78,
 62.86,
 62.94,
 63.03,
 63.11,
 63.19,
 63.27,
 63.36,
 63.44,
 63.52,
 63.61,
 63.69,
 63.77,
 63.85,
 63.94,
 64.02,
 64.1,
 64.18,
 64.27,
 64.35,
 64.43,
 64.52,
 64.6,
 64.68,
 64.76,
 64.85,
 64.93,
 65.01,
 65.1,
 65.18,
 65.26,
 65.34,
 65.43,
 65.51,
 65.59,
 65.67,
 65.76,
 65.84,
 65.92,
 66.01,
 66.09,
 66.17,
 66.25,
 66.34,
 66.42,
 66.5,
 66.58,
 66.67,
 66.75,
 66.83,
 66.92,
 67.0,
 67.08,
 67.16,
 67.25,
 67.33,
 67.41,
 67.49,
 67.58,
 67.66,
 67.74,
 67.83,
 67.91,
 67.99,
 68.07,
 68.16,
 68.2

In [12]:
# for weight graph
graph_weight = [input_weight] # 0일 뒤 몸무게 == 현재 입력 몸무게
x_weight_days = list(range(1,181)) # 1일 뒤부터 180일 뒤 까지
for i in range(1,181):
    graph_weight.append(round(lr_weight.predict([[input_days + i, gender, graph_height[i]]])[0], 2))
graph_weight


[4.8,
 5.41,
 5.43,
 5.45,
 5.47,
 5.5,
 5.52,
 5.54,
 5.56,
 5.59,
 5.61,
 5.63,
 5.65,
 5.68,
 5.7,
 5.72,
 5.74,
 5.77,
 5.79,
 5.81,
 5.83,
 5.85,
 5.88,
 5.9,
 5.92,
 5.94,
 5.97,
 5.99,
 6.01,
 6.03,
 6.06,
 6.08,
 6.1,
 6.12,
 6.15,
 6.17,
 6.19,
 6.22,
 6.24,
 6.26,
 6.28,
 6.31,
 6.33,
 6.35,
 6.37,
 6.4,
 6.42,
 6.44,
 6.46,
 6.49,
 6.51,
 6.53,
 6.55,
 6.58,
 6.6,
 6.62,
 6.64,
 6.67,
 6.69,
 6.71,
 6.73,
 6.76,
 6.78,
 6.8,
 6.82,
 6.85,
 6.87,
 6.89,
 6.91,
 6.93,
 6.96,
 6.98,
 7.0,
 7.02,
 7.05,
 7.07,
 7.09,
 7.11,
 7.14,
 7.16,
 7.18,
 7.21,
 7.23,
 7.25,
 7.27,
 7.3,
 7.32,
 7.34,
 7.36,
 7.39,
 7.41,
 7.43,
 7.45,
 7.48,
 7.5,
 7.52,
 7.54,
 7.57,
 7.59,
 7.61,
 7.63,
 7.66,
 7.68,
 7.7,
 7.72,
 7.75,
 7.77,
 7.79,
 7.81,
 7.84,
 7.86,
 7.88,
 7.9,
 7.92,
 7.95,
 7.97,
 7.99,
 8.01,
 8.04,
 8.06,
 8.08,
 8.11,
 8.13,
 8.15,
 8.17,
 8.2,
 8.22,
 8.24,
 8.26,
 8.29,
 8.31,
 8.33,
 8.35,
 8.38,
 8.4,
 8.42,
 8.44,
 8.47,
 8.49,
 8.51,
 8.53,
 8.56,
 8.58,
 8.6,
 8.62,
 